In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
#from quantile_forest import RandomForestQuantileRegressor
#from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/prediction-interval-competition-ii-house-price/sample_submission.csv
/kaggle/input/prediction-interval-competition-ii-house-price/test.csv
/kaggle/input/prediction-interval-competition-ii-house-price/dataset.csv


In [2]:
train = pd.read_csv('/kaggle/input/prediction-interval-competition-ii-house-price/dataset.csv', parse_dates=['sale_date'], index_col='id')
test = pd.read_csv('/kaggle/input/prediction-interval-competition-ii-house-price/test.csv', parse_dates=['sale_date'], index_col='id')

In [3]:
train.shape

(200000, 46)

In [4]:
test.shape

(200000, 45)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200000 entries, 0 to 199999
Data columns (total 46 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   sale_date         200000 non-null  datetime64[ns]
 1   sale_price        200000 non-null  int64         
 2   sale_nbr          157818 non-null  float64       
 3   sale_warning      200000 non-null  object        
 4   join_status       200000 non-null  object        
 5   join_year         200000 non-null  int64         
 6   latitude          200000 non-null  float64       
 7   longitude         200000 non-null  float64       
 8   area              200000 non-null  int64         
 9   city              200000 non-null  object        
 10  zoning            200000 non-null  object        
 11  subdivision       182450 non-null  object        
 12  present_use       200000 non-null  int64         
 13  land_val          200000 non-null  int64         
 14  imp_val  

In [6]:
train.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,sale_date,sale_price,sale_nbr,sale_warning,join_status,join_year,latitude,longitude,area,city,zoning,subdivision,present_use,land_val,imp_val,year_built,year_reno,sqft_lot,sqft,sqft_1,sqft_fbsmt,grade,fbsmt_grade,condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2014-11-15,236000,2.0,,nochg,2025,47.2917,-122.3658,53,FEDERAL WAY,RS7.2,ALDERWOOD SOUTH DIV NO. 02,2,167000,372000,1975,0,10919,1560,1560,0,7,0,5,1.0,3,1,1,0,0,500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,I
1,1999-01-15,313300,NaN,26,nochg,2025,47.6531,-122.1996,74,KIRKLAND,RS 8.5,WILDWOOD LANE NO. 03,2,1184000,598000,1962,0,8900,2040,1220,820,7,7,4,1.0,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Q
2,2006-08-15,341000,1.0,,nochg,2025,47.4733,-122.1901,30,RENTON,R-8,FALCON RIDGE (CEDAR RIDGE),2,230000,356000,1986,0,4953,1640,820,0,7,0,3,2.0,3,2,0,1,0,480,0,0,0,0,0,0,0,0,0,0,0,0,0,0,K
3,1999-12-15,267000,1.0,,nochg,2025,47.4739,-122.3295,96,BURIEN,RS-7200,OLYMPIC VUE ESTATES,2,190000,518000,1998,0,6799,2610,1010,500,8,7,3,2.0,4,2,0,1,0,530,0,0,0,1,0,0,0,0,0,0,0,0,0,0,G
4,2018-07-15,1650000,2.0,,miss99,2025,47.7516,-122.1222,36,KING COUNTY,RA2.5,HOLLYWOOD HILL HIGHLANDS,2,616000,1917000,1998,0,31687,4040,3640,0,12,0,3,2.0,4,2,1,1,0,810,0,0,0,0,0,0,0,0,0,0,0,0,0,0,P


In [7]:
test.head()

,sale_date,sale_nbr,sale_warning,join_status,join_year,latitude,longitude,area,city,zoning,subdivision,present_use,land_val,imp_val,year_built,year_reno,sqft_lot,sqft,sqft_1,sqft_fbsmt,grade,fbsmt_grade,condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
200000,2022-12-15,4.0,,new,2025,47.4794,-122.2212,25,RENTON,R-8,TOWN-COUNTRY CLUB BUNGALOW SITES,2,320000,588000,2019,0,4127,2770,1130,0,8,0,3,2.0,4,1,1,1,0,480,0,0,0,0,0,0,0,0,0,0,0,0,0,0,J
200001,2000-10-15,2.0,,new,2025,47.5778,-122.0270,69,SAMMAMISH,R1,NaN,2,491000,1302000,2000,0,29029,3570,1750,0,11,0,3,2.0,4,3,0,0,0,920,0,0,0,1,0,0,0,0,0,0,0,0,0,0,O
200002,2004-06-15,1.0,,nochg,2025,47.5632,-122.1623,31,BELLEVUE,R-3.5,SOMERSET NO. 08,2,1292000,403000,1967,0,8700,1900,1900,0,8,0,4,1.0,4,1,1,0,0,510,0,0,0,0,0,3,0,3,3,0,2,0,0,0,R
200003,2009-11-15,1.0,,nochg,2025,47.6911,-122.3469,6,SEATTLE,LR1 (M1),NEVINS DIV OF GREEN LAKE ADD,2,500000,291000,1965,0,5100,1980,1270,710,7,6,3,1.0,3,1,1,0,480,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,B
200004,1999-06-15,1.0,,reno - before,1999,47.4001,-122.4146,100,KING COUNTY,RA2.5P,NaN,2,124000,277000,1920,1983,47480,3150,990,960,8,6,3,1.5,3,1,2,0,0,0,3,0,0,0,0,0,2,4,0,3,0,0,0,0,H


In [8]:
train.isnull().sum()

sale_date               0
sale_price              0
sale_nbr            42182
sale_warning            0
join_status             0
join_year               0
latitude                0
longitude               0
area                    0
city                    0
zoning                  0
subdivision         17550
present_use             0
land_val                0
imp_val                 0
year_built              0
year_reno               0
sqft_lot                0
sqft                    0
sqft_1                  0
sqft_fbsmt              0
grade                   0
fbsmt_grade             0
condition               0
stories                 0
beds                    0
bath_full               0
bath_3qtr               0
bath_half               0
garb_sqft               0
gara_sqft               0
wfnt                    0
golf                    0
greenbelt               0
noise_traffic           0
view_rainier            0
view_olympics           0
view_cascades           0
view_territo

In [9]:
test.isnull().sum()

sale_date               0
sale_nbr            42412
sale_warning            0
join_status             0
join_year               0
latitude                0
longitude               0
area                    0
city                    0
zoning                  0
subdivision         17550
present_use             0
land_val                0
imp_val                 0
year_built              0
year_reno               0
sqft_lot                0
sqft                    0
sqft_1                  0
sqft_fbsmt              0
grade                   0
fbsmt_grade             0
condition               0
stories                 0
beds                    0
bath_full               0
bath_3qtr               0
bath_half               0
garb_sqft               0
gara_sqft               0
wfnt                    0
golf                    0
greenbelt               0
noise_traffic           0
view_rainier            0
view_olympics           0
view_cascades           0
view_territorial        0
view_skyline

In [10]:
train.shape

(200000, 46)

In [11]:
cat_cols = train.select_dtypes(include=['object']).columns

for col in cat_cols:
    train[col] = train[col].fillna(train[col].mode()[0])
    test[col] = test[col].fillna(test[col].mode()[0])

In [12]:
from category_encoders import CountEncoder

ce = CountEncoder()
train[cat_cols] = ce.fit_transform(train[cat_cols])
test[cat_cols] = ce.transform(test[cat_cols])

In [13]:
train['sale_nbr'] = train['sale_nbr'].fillna(train['sale_nbr'].median())
test['sale_nbr'] = test['sale_nbr'].fillna(test['sale_nbr'].median())

In [14]:
train.isnull().sum()

sale_date           0
sale_price          0
sale_nbr            0
sale_warning        0
join_status         0
join_year           0
latitude            0
longitude           0
area                0
city                0
zoning              0
subdivision         0
present_use         0
land_val            0
imp_val             0
year_built          0
year_reno           0
sqft_lot            0
sqft                0
sqft_1              0
sqft_fbsmt          0
grade               0
fbsmt_grade         0
condition           0
stories             0
beds                0
bath_full           0
bath_3qtr           0
bath_half           0
garb_sqft           0
gara_sqft           0
wfnt                0
golf                0
greenbelt           0
noise_traffic       0
view_rainier        0
view_olympics       0
view_cascades       0
view_territorial    0
view_skyline        0
view_sound          0
view_lakewash       0
view_lakesamm       0
view_otherwater     0
view_other          0
submarket 

In [15]:
train.head()

,sale_date,sale_price,sale_nbr,sale_warning,join_status,join_year,latitude,longitude,area,city,zoning,subdivision,present_use,land_val,imp_val,year_built,year_reno,sqft_lot,sqft,sqft_1,sqft_fbsmt,grade,fbsmt_grade,condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2014-11-15,236000,2.0,180689,126281,2025,47.2917,-122.3658,53,7616,5247,13,2,167000,372000,1975,0,10919,1560,1560,0,7,0,5,1.0,3,1,1,0,0,500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15842
1,1999-01-15,313300,2.0,6831,126281,2025,47.6531,-122.1996,74,8563,1096,3,2,1184000,598000,1962,0,8900,2040,1220,820,7,7,4,1.0,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,14005
2,2006-08-15,341000,1.0,180689,126281,2025,47.4733,-122.1901,30,9368,4733,61,2,230000,356000,1986,0,4953,1640,820,0,7,0,3,2.0,3,2,0,1,0,480,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22719
3,1999-12-15,267000,1.0,180689,126281,2025,47.4739,-122.3295,96,3823,4558,5,2,190000,518000,1998,0,6799,2610,1010,500,8,7,3,2.0,4,2,0,1,0,530,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6144
4,2018-07-15,1650000,2.0,180689,2468,2025,47.7516,-122.1222,36,27127,2454,5,2,616000,1917000,1998,0,31687,4040,3640,0,12,0,3,2.0,4,2,1,1,0,810,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6971


In [16]:
class SaleDateEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, date_column='sale_date'):
        self.date_column = date_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.assign(
            **{
                'sale_week': lambda x: x['sale_date'].dt.isocalendar().week,
                'month': lambda x: x['sale_date'].dt.month,
                'year': lambda x: x['sale_date'].dt.year,
                'day': lambda x: x['sale_date'].dt.weekday,
            }
        ).drop(columns=['sale_date'])
        return X

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
  

In [17]:
encoder = SaleDateEncoder(date_column='sale_date')
train = encoder.fit_transform(train)
test = encoder.transform(test)

In [18]:
train.head()

,sale_price,sale_nbr,sale_warning,join_status,join_year,latitude,longitude,area,city,zoning,subdivision,present_use,land_val,imp_val,year_built,year_reno,sqft_lot,sqft,sqft_1,sqft_fbsmt,grade,fbsmt_grade,condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket,sale_week,month,year,day
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,236000,2.0,180689,126281,2025,47.2917,-122.3658,53,7616,5247,13,2,167000,372000,1975,0,10919,1560,1560,0,7,0,5,1.0,3,1,1,0,0,500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15842,46,11,2014,5
1,313300,2.0,6831,126281,2025,47.6531,-122.1996,74,8563,1096,3,2,1184000,598000,1962,0,8900,2040,1220,820,7,7,4,1.0,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,14005,2,1,1999,4
2,341000,1.0,180689,126281,2025,47.4733,-122.1901,30,9368,4733,61,2,230000,356000,1986,0,4953,1640,820,0,7,0,3,2.0,3,2,0,1,0,480,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22719,33,8,2006,1
3,267000,1.0,180689,126281,2025,47.4739,-122.3295,96,3823,4558,5,2,190000,518000,1998,0,6799,2610,1010,500,8,7,3,2.0,4,2,0,1,0,530,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6144,50,12,1999,2
4,1650000,2.0,180689,2468,2025,47.7516,-122.1222,36,27127,2454,5,2,616000,1917000,1998,0,31687,4040,3640,0,12,0,3,2.0,4,2,1,1,0,810,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6971,28,7,2018,6


In [19]:
alpha = 0.1

In [20]:
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    
    y_true = np.asarray(y_true)
    lower = np.asarray(lower)
    upper = np.asarray(upper)

    width = upper - lower
    penalty_lower = 2/ alpha * (lower - y_true)
    penalty_upper = 2/ alpha * (y_true - upper)

    score = width.copy()
    score += np.where(y_true < lower, penalty_lower, 0)
    score += np.where(y_true > upper, penalty_upper, 0)

    if return_coverage:
        inside = (y_true >= lower) & (y_true <= upper)
        coverage = np.mean(inside)
        return np.mean(score), coverage

    return np.mean(score)

In [21]:
from sklearn.model_selection import train_test_split

X = train.copy()
y = X.pop('sale_price')

X_test = test.copy()

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.55, random_state=30)

In [22]:
params = {
    'n_estimators': 300,
    'max_depth': 4,
    'learning_rate': 0.05
}

In [23]:
lower_model = XGBRegressor(
    objective="reg:quantileerror", 
    quantile_alpha=0.05,
    **params,
).fit(X_train, y_train)

pred_lower = lower_model.predict(X_valid)

upper_model = XGBRegressor(
    objective="reg:quantileerror", 
    quantile_alpha=0.95,
    **params,
).fit(X_train, y_train)

pred_upper = upper_model.predict(X_valid)

score = winkler_score(y_valid, pred_lower, pred_upper, alpha)
print(score)

407784.75


In [24]:
X.head()

,sale_nbr,sale_warning,join_status,join_year,latitude,longitude,area,city,zoning,subdivision,present_use,land_val,imp_val,year_built,year_reno,sqft_lot,sqft,sqft_1,sqft_fbsmt,grade,fbsmt_grade,condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket,sale_week,month,year,day
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2.0,180689,126281,2025,47.2917,-122.3658,53,7616,5247,13,2,167000,372000,1975,0,10919,1560,1560,0,7,0,5,1.0,3,1,1,0,0,500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15842,46,11,2014,5
1,2.0,6831,126281,2025,47.6531,-122.1996,74,8563,1096,3,2,1184000,598000,1962,0,8900,2040,1220,820,7,7,4,1.0,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,14005,2,1,1999,4
2,1.0,180689,126281,2025,47.4733,-122.1901,30,9368,4733,61,2,230000,356000,1986,0,4953,1640,820,0,7,0,3,2.0,3,2,0,1,0,480,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22719,33,8,2006,1
3,1.0,180689,126281,2025,47.4739,-122.3295,96,3823,4558,5,2,190000,518000,1998,0,6799,2610,1010,500,8,7,3,2.0,4,2,0,1,0,530,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6144,50,12,1999,2
4,2.0,180689,2468,2025,47.7516,-122.1222,36,27127,2454,5,2,616000,1917000,1998,0,31687,4040,3640,0,12,0,3,2.0,4,2,1,1,0,810,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6971,28,7,2018,6


In [25]:
lower_model = XGBRegressor(
    objective="reg:quantileerror", 
    quantile_alpha=0.05,
    **params,
).fit(X, y)

upper_model = XGBRegressor(
    objective="reg:quantileerror", 
    quantile_alpha=0.95,
    **params,
).fit(X, y)

In [26]:
lower_pi = lower_model.predict(X_test)
upper_pi = upper_model.predict(X_test)

In [27]:
sub = pd.read_csv('/kaggle/input/prediction-interval-competition-ii-house-price/sample_submission.csv')
sub['pi_lower'] = lower_pi
sub['pi_upper'] = upper_pi
sub.to_csv('submission.csv', index=False)
sub.head(10)


,id,pi_lower,pi_upper
0,200000,809937.93750,1.160132e+06
1,200001,513416.06250,7.983619e+05
2,200002,442627.03125,7.582430e+05
3,200003,306918.75000,4.628276e+05
4,200004,307663.46875,1.119768e+06
5,200005,638740.93750,9.553323e+05
6,200006,386311.53125,6.544714e+05
7,200007,315454.71875,4.408153e+05
8,200008,372604.18750,5.163041e+05
9,200009,342077.03125,6.557478e+05
